In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
from tqdm import tqdm
import os
import re
import time
import pickle

pd.set_option('display.max_columns', 200)

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
tf1 = tf.compat.v1
tf1.disable_eager_execution()
tf1.disable_v2_behavior()

2025-03-03 02:27:43.056391: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-03 02:27:43.069564: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-03 02:27:43.183504: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-03 02:27:43.316476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740949063.435922    6839 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740949063.46

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:

from cachetools import TTLCache,cached
cache = TTLCache(maxsize=100, ttl=21600)

In [4]:
df_train = pd.read_csv('/home/smrutibiswal/Research/TextClassificationProject/Data/train_2kmZucJ.csv')
df_test = pd.read_csv('Data/test_oJQbWVk.csv')

print(df_train.shape)
print(df_test.shape)


(7920, 3)
(1953, 2)


In [5]:
df_train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [6]:
df_train['label'].value_counts(normalize=True)

label
0    0.744192
1    0.255808
Name: proportion, dtype: float64

In [7]:
df_train['clean_tweet'] = df_train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
df_test['clean_tweet'] = df_test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
df_train.head()

,id,label,tweet,clean_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,#fingerprint #Pregnancy Test #android #apps #...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...,What amazing service! Apple won't even talk to...


In [8]:
def remove_hashtags_mentions(input_txt):
    input_txt = re.sub(r'#', '', input_txt)
    input_txt = re.sub(r'@', '', input_txt)
    input_txt = re.sub(r'[^\w\s]', '', input_txt)
    # make all lower case
    input_txt = input_txt.lower()
    
    return input_txt


In [9]:
df_train['clean_tweet'] = df_train['clean_tweet'].apply(remove_hashtags_mentions)
df_test['clean_tweet'] = df_test['clean_tweet'].apply(remove_hashtags_mentions)
df_train.head()

,id,label,tweet,clean_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,fingerprint pregnancy test android apps beaut...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,finally a transparant silicon case thanks to ...
2,3,0,We love this! Would you go? #talk #makememorie...,we love this would you go talk makememories un...
3,4,0,I'm wired I know I'm George I was made that wa...,im wired i know im george i was made that way ...
4,5,1,What amazing service! Apple won't even talk to...,what amazing service apple wont even talk to m...


In [10]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [11]:
def lemmatization(texts):
    output = []
    for word in texts:
        s = [token.lemma_ for token in nlp(word)]
        output.append(' '.join(s))
    return output


In [12]:
df_train['clean_tweet'] = lemmatization(df_train['clean_tweet'])
df_test['clean_tweet'] = lemmatization(df_test['clean_tweet'])

df_train.sample(10)

,id,label,tweet,clean_tweet
1916,1917,0,"#new #apps for #iPhone, for the #people who #s...",new app for iphone for the people who sail in ...
5616,5617,1,Not! #hateApple RT @npolowydraws This Father's...,not hateapple rt npolowydraw this father day y...
4302,4303,0,My City..My View..NYC #ny #nyc #newyork #usa #...,my citymy viewnyc ny nyc newyork usa timesq tr...
5910,5911,0,Morning selfie #me #model #igers #iphone #igda...,morning selfie I model iger iphone igdaily ins...
4294,4295,0,Let's Call it art #amazing #beautiful #copenha...,let call it art amazing beautiful copenhagen c...
1434,1435,0,Mommy - Daughter day! Wicked! Broadway fun! #w...,mommy daughter day wicked broadway fun wicke...
3588,3589,0,#Verizon will announce the #iPhone 4 tomorrow ...,verizon will announce the iphone 4 tomorrow ...
4993,4994,0,Reposting @ksr.design: Gin and juice! #custome...,reposte ksrdesign gin and juice customer custo...
4538,4539,1,Apple Is Sorry About All That Calendar Spam ht...,apple be sorry about all that calendar spam ...
3574,3575,1,@hgomez and of course the university program i...,hgomez and of course the university program be...


## Elmo


In [13]:
y_train = tf.keras.utils.to_categorical(df_train.label.values)

In [14]:
# elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/2", trainable=False)
# elmo(tf.constant(["hello world, how are you?"]))

# embeddings = elmo(tf.constant(["apple pineapple orange juice", "apple pineapple juice"]))
# embeddings.shape

# def ElmoEmbedding(x):
#     return elmo(tf.reshape(tf.cast(x, tf.string), [-1]))

# text = ["apple pineapple orange juice", "apple pineapple juice"]
# embeddings = ElmoEmbedding(text)
# embeddings.shape


In [15]:
elmo = hub.load("https://tfhub.dev/google/elmo/2")

2025-03-03 02:28:04.000399: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [16]:
text = ["apple pineapple orange juice", "apple pineapple juice"]
embeddings = elmo.signatures["default"](tf.constant(text))["elmo"]
embeddings.shape

TensorShape([Dimension(2), Dimension(4), Dimension(1024)])

In [17]:
@cached(cache)
def ElmoEmbedding(x):
    return elmo.signatures["default"](tf.reshape(tf.cast(x,tf.string)),[-1])["elmo"]

In [18]:
def create_model_architecture():
    input_text = tf.keras.layers.Input(shape=(), dtype='string', name='input_text')  # Remove (1,)

    embedding = tf.keras.layers.Lambda(lambda x: tf.expand_dims(ElmoEmbedding(x), axis=1),
                                       output_shape=(1, 1024), name='elmo_embedding')(input_text)

    x = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(1024, return_sequences=False, recurrent_dropout=0.2, dropout=0.2, name="BiLSTM")
    )(embedding)

    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(2, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_text], outputs=predictions)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model



In [19]:
model = create_model_architecture()

In [20]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [21]:
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    sess.run(tf.compat.v1.tables_initializer())
    # history = model.fit(np.asarray(df_train.clean_tweet),y_train,epochs=5,batch_size = 2,validation_split=0.2)
    history = model.fit(np.asarray(df_train.clean_tweet),np.asarray(df_train.label.values),epochs=3,batch_size = 32,validation_split=0.2)
    model.save_weights('../models/model_elmo_weights.h5')

I0000 00:00:1740949084.885321    6839 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-03-03 02:28:08.796107: W tensorflow/c/c_api.cc:305] Operation '{name:'PartitionedCall' id:82 op device:{requested: '', assigned: ''} def:{{{node PartitionedCall}} = PartitionedCall[Tin=[DT_RESOURCE, DT_RESOURCE, DT_RESOURCE, DT_RESOURCE, DT_RESOURCE, ..., DT_RESOURCE, DT_RESOURCE, DT_RESOURCE, DT_RESOURCE, DT_RESOURCE]{attr_hash=13055791533631553071}, Tout=[DT_FLOAT], _collective_manager_ids=[], _has_manual_control_dependencies=true, _read_only_resource_inputs=[0, 1, 2, 3, 4, ..., 34, 35, 36, 37, 38]{attr_hash=5705907006345926617}, config="", config_proto="\n\007\n\0...1\002J\000", executor_type="", f=__inference_pruned_1991[]](bilm/char_embed, bilm/CNN/W_cnn_0, bilm/CNN/b_cnn_0, bilm/CNN/W_cnn_1, bilm/CNN/b_cnn_1, bilm/CNN/W_cnn_2, bilm/CNN/b_cnn_2, bilm/CNN/W_cnn_3, bilm/CNN/b_cnn_3, bilm/CNN/W_cnn_4, bilm/CNN/b_cnn_4, bilm/CNN/W_cnn_5, bilm/CNN/b_cnn_5, bilm/CNN/W

TypeError: unhashable type: 'Dimension'

In [ ]:
print(history.history.keys())